# BPNN training and testing 

In [1]:
import torch
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import random
from torch.utils.data import Dataset, DataLoader
from Model import Net
from trainer import Trainer
import dataloader


Starting by creating options for training, testing and usage.

In [12]:
class Args:
    label_dir = "./Label.csv"
    image_dir = "./ROI_trab"
    batch_size = 8
    nof = 16
    lr = 0.001
    nb_epochs = 5
    checkpoint_path = "./"
    mode = "Training"
    cross_val = False
    k_fold = 5


opt = Args()

NB_LABEL = 14
PERCENTAGE_TEST = 20
RESIZE_IMAGE = 512

In [13]:
model = Net()

Dataset creation from label and images

In [14]:
datasets = dataloader.Datasets(csv_file = opt.label_dir, image_dir = opt.image_dir)

Here, let's create our cross validation dataset

In [15]:
kf = KFold(n_splits = opt.k_fold, shuffle=True)
kf.get_n_splits(datasets)

5

In [16]:
for train_index, test_index in kf.split(datasets):
    print("Train:", train_index[1:4],"Test:",test_index[1:4])
    nb_data = len(datasets)
    trainloader = DataLoader(datasets, batch_size = opt.batch_size, sampler = train_index,  num_workers = 0 )
    testloader =DataLoader(datasets, batch_size = 1, sampler = test_index, num_workers = 0 )
    t = Trainer(opt,model)
    for epoch in range(opt.nb_epochs):
        t.train(trainloader,epoch)
        t.test(testloader,epoch)
        

Train: [1 2 4] Test: [ 5  8 11]
starting training
----------------
[1     8], loss: 6874795.508


KeyboardInterrupt: 